# Generators and Iterators

[Click here to run this chapter on Colab](https://colab.research.google.com/github/AllenDowney/DSIRP/blob/main/notebooks/generator.ipynb)

This chapter introduces generator functions, which are functions that yield a stream of values, rather than returning a single value.

To demonstrate their use, we'll explore Cartesian products, permutations, and combinations, using playing cards as an example.

## Generators

As a first example, we'll write a generator function that generates the playing cards in a standard 52-card deck.
This example is inspired by an example in Peter Norvig's ["A Concrete Introduction to Probability (using Python)"](https://nbviewer.ipython.org/url/norvig.com/ipython/Probability.ipynb).

Here are Unicode strings that represent the set of suits and the set of ranks.

In [2]:
suits = u'♠♥♦♣'
ranks = u'AKQJ⑽98765432'

And here's a nested for loop that enumerates all pairings of a rank with a suit.

In [3]:
for rank in ranks:
    for suit in suits:
        print(rank+suit, end=' ')

A♠ A♥ A♦ A♣ K♠ K♥ K♦ K♣ Q♠ Q♥ Q♦ Q♣ J♠ J♥ J♦ J♣ ⑽♠ ⑽♥ ⑽♦ ⑽♣ 9♠ 9♥ 9♦ 9♣ 8♠ 8♥ 8♦ 8♣ 7♠ 7♥ 7♦ 7♣ 6♠ 6♥ 6♦ 6♣ 5♠ 5♥ 5♦ 5♣ 4♠ 4♥ 4♦ 4♣ 3♠ 3♥ 3♦ 3♣ 2♠ 2♥ 2♦ 2♣ 

This set of pairs is the [Cartesian product](https://en.wikipedia.org/wiki/Cartesian_product) of the set of ranks and the set of suits.

The following function encapsulates the loops and uses the `yield` statement to generate a stream of cards.

In [4]:
def card_generator(ranks, suits):
    for rank in ranks:
        for suit in suits:
            yield rank+suit

Because this function includes a `yield` statement, it is a generator function. When we call it, the return value is a generator object.

In [5]:
it = card_generator(ranks, suits)
it

<generator object card_generator at 0x7efb7460fb50>

The generator object is iterable, so we can use `next` to get the first element of the stream.

In [6]:
next(it)

'A♠'

The first time we call `next`, the function runs until it hits the `yield` statement.
If we call `next` again, the function resumes from where it left off and runs until it hits the `yield` statement again.

In [ ]:
next(it)

Because `it` is iterable, we can use it in a for loop to enumerate the remaining pairs.

In [7]:
for card in it:
    print(card, end=' ')

A♥ A♦ A♣ K♠ K♥ K♦ K♣ Q♠ Q♥ Q♦ Q♣ J♠ J♥ J♦ J♣ ⑽♠ ⑽♥ ⑽♦ ⑽♣ 9♠ 9♥ 9♦ 9♣ 8♠ 8♥ 8♦ 8♣ 7♠ 7♥ 7♦ 7♣ 6♠ 6♥ 6♦ 6♣ 5♠ 5♥ 5♦ 5♣ 4♠ 4♥ 4♦ 4♣ 3♠ 3♥ 3♦ 3♣ 2♠ 2♥ 2♦ 2♣ 

When the flow of control reaches the end of the function, the generator object raises and exception, which causes the for loop to end.

## itertools

The `itertools` library provides function for working with iterators, including `product`, which is a generator function that takes iterators as arguments at yields their Cartesian product.
We'll use `itertools.product` in the next few sections; then we'll see how to implement it.

Here's a loop that uses `itertools.product` to generate the playing cards again.

In [8]:
from itertools import product

for t in product(ranks, suits):
    card = ''.join(t)
    print(card, end=' ')

A♠ A♥ A♦ A♣ K♠ K♥ K♦ K♣ Q♠ Q♥ Q♦ Q♣ J♠ J♥ J♦ J♣ ⑽♠ ⑽♥ ⑽♦ ⑽♣ 9♠ 9♥ 9♦ 9♣ 8♠ 8♥ 8♦ 8♣ 7♠ 7♥ 7♦ 7♣ 6♠ 6♥ 6♦ 6♣ 5♠ 5♥ 5♦ 5♣ 4♠ 4♥ 4♦ 4♣ 3♠ 3♥ 3♦ 3♣ 2♠ 2♥ 2♦ 2♣ 

**Exercise:** Encapsulate the previous loop in a generator function called `card_generator2` that yields the playing  cards. Then call your function and use it to print the cards.

## Enumerating all pairs

Now that we have playing cards, let's deal a few hands. In fact, let's deal all the hands.

First, I'll create two card generators.

In [9]:
it1 = card_generator(ranks, suits)
it2 = card_generator(ranks, suits)

Now we can use `product` to generate all pairs of cards.

In [ ]:
for hand in product(it1, it2):
    print(hand)

To check whether it's working correctly, it will be useful to count the number of elements in an iterator, which is what `ilen` does.
This idiom is discussed [on Stack Overflow](https://stackoverflow.com/questions/390852/is-there-any-built-in-way-to-get-the-length-of-an-iterable-in-python).

In [10]:
def ilen(it):
    return sum(1 for _ in it)

Now we can use it to count the pairs of cards.

In [11]:
it1 = card_generator(ranks, suits)
it2 = card_generator(ranks, suits)
ilen(product(it1, it2))

2704

If things have gone according to plan, the number of pairs should be $52^2$.

In [ ]:
52**2

Notice that we have to create new card iterators every time, because once they are used up, they behave like an empty list.
Here's what happens if we try to use them again.

In [12]:
ilen(product(it1, it2))

0

That's also why we had to create two card iterators.
If you create one and try to use it twice, it doesn't work.

In [13]:
it = card_generator(ranks, suits)
ilen(product(it, it))

0

However, you can get around this limitation by calling `product` with the `repeat` argument, which makes it possible to use a single iterator to generate a Cartesian product.

In [ ]:
it = card_generator(ranks, suits)
ilen(product(it, repeat=2))

## Permutations

In the previous section, you might have noticed that some of the hands we generated are impossible because they contain the same card more than once.

One way to solve this problem is to generate all pairs and then eliminate the ones that contain duplicates.

In [14]:
it = card_generator(ranks, suits)

for hand in product(it, repeat=2):
    if len(hand) == len(set(hand)):
        print(hand)

('A♠', 'A♥')
('A♠', 'A♦')
('A♠', 'A♣')
('A♠', 'K♠')
('A♠', 'K♥')
('A♠', 'K♦')
('A♠', 'K♣')
('A♠', 'Q♠')
('A♠', 'Q♥')
('A♠', 'Q♦')
('A♠', 'Q♣')
('A♠', 'J♠')
('A♠', 'J♥')
('A♠', 'J♦')
('A♠', 'J♣')
('A♠', '⑽♠')
('A♠', '⑽♥')
('A♠', '⑽♦')
('A♠', '⑽♣')
('A♠', '9♠')
('A♠', '9♥')
('A♠', '9♦')
('A♠', '9♣')
('A♠', '8♠')
('A♠', '8♥')
('A♠', '8♦')
('A♠', '8♣')
('A♠', '7♠')
('A♠', '7♥')
('A♠', '7♦')
('A♠', '7♣')
('A♠', '6♠')
('A♠', '6♥')
('A♠', '6♦')
('A♠', '6♣')
('A♠', '5♠')
('A♠', '5♥')
('A♠', '5♦')
('A♠', '5♣')
('A♠', '4♠')
('A♠', '4♥')
('A♠', '4♦')
('A♠', '4♣')
('A♠', '3♠')
('A♠', '3♥')
('A♠', '3♦')
('A♠', '3♣')
('A♠', '2♠')
('A♠', '2♥')
('A♠', '2♦')
('A♠', '2♣')
('A♥', 'A♠')
('A♥', 'A♦')
('A♥', 'A♣')
('A♥', 'K♠')
('A♥', 'K♥')
('A♥', 'K♦')
('A♥', 'K♣')
('A♥', 'Q♠')
('A♥', 'Q♥')
('A♥', 'Q♦')
('A♥', 'Q♣')
('A♥', 'J♠')
('A♥', 'J♥')
('A♥', 'J♦')
('A♥', 'J♣')
('A♥', '⑽♠')
('A♥', '⑽♥')
('A♥', '⑽♦')
('A♥', '⑽♣')
('A♥', '9♠')
('A♥', '9♥')
('A♥', '9♦')
('A♥', '9♣')
('A♥', '8♠')
('A♥', '8♥')
('A♥', '8♦')

**Exercise:** Write a generator function called `permutations` that takes an iterator and and integer, `r`, as arguments. It should generate tuples that represent all subsets of the elements in the iterator with size `r` and no duplicates.

Test your function by generating and printing all hands with two distinct cards.
Then use `ilen` to count how many there are, and confirm that it's `52 * 51`.

In [20]:
def permutations(it, r):
  for hand in product(it, repeat=r):
    if len(hand) == len(set(hand)):
      yield hand

The `itertools` library provides a function called `permutations` that does the same thing.

In [15]:
import itertools

it = card_generator(ranks, suits)
ilen(itertools.permutations(it, 2))

2652

## Combinations

At this point we are generating legitimate hands in the sense that the same card never appears twice.
But we end up generating the same hand more than once, in the sense that the order of the cards does not matter.
So we consider `(card1, card2)` to be the same hand as `(card2, card1)`.
To avoid that, we can generate all permutations and then filter out the ones that are not in sorted order.

It doesn't really matter which order is considered "sorted"; it's just a way to choose one ordering we consider "canonical".

That's what the following loop does.

In [21]:
it = card_generator(ranks, suits)

for hand in permutations(it, r=2):
    if list(hand) == sorted(hand):
        print(hand)

('A♠', 'A♥')
('A♠', 'A♦')
('A♠', 'A♣')
('A♠', 'K♠')
('A♠', 'K♥')
('A♠', 'K♦')
('A♠', 'K♣')
('A♠', 'Q♠')
('A♠', 'Q♥')
('A♠', 'Q♦')
('A♠', 'Q♣')
('A♠', 'J♠')
('A♠', 'J♥')
('A♠', 'J♦')
('A♠', 'J♣')
('A♠', '⑽♠')
('A♠', '⑽♥')
('A♠', '⑽♦')
('A♠', '⑽♣')
('A♥', 'A♦')
('A♥', 'K♠')
('A♥', 'K♥')
('A♥', 'K♦')
('A♥', 'K♣')
('A♥', 'Q♠')
('A♥', 'Q♥')
('A♥', 'Q♦')
('A♥', 'Q♣')
('A♥', 'J♠')
('A♥', 'J♥')
('A♥', 'J♦')
('A♥', 'J♣')
('A♥', '⑽♠')
('A♥', '⑽♥')
('A♥', '⑽♦')
('A♥', '⑽♣')
('A♦', 'K♠')
('A♦', 'K♥')
('A♦', 'K♦')
('A♦', 'K♣')
('A♦', 'Q♠')
('A♦', 'Q♥')
('A♦', 'Q♦')
('A♦', 'Q♣')
('A♦', 'J♠')
('A♦', 'J♥')
('A♦', 'J♦')
('A♦', 'J♣')
('A♦', '⑽♠')
('A♦', '⑽♥')
('A♦', '⑽♦')
('A♦', '⑽♣')
('A♣', 'A♥')
('A♣', 'A♦')
('A♣', 'K♠')
('A♣', 'K♥')
('A♣', 'K♦')
('A♣', 'K♣')
('A♣', 'Q♠')
('A♣', 'Q♥')
('A♣', 'Q♦')
('A♣', 'Q♣')
('A♣', 'J♠')
('A♣', 'J♥')
('A♣', 'J♦')
('A♣', 'J♣')
('A♣', '⑽♠')
('A♣', '⑽♥')
('A♣', '⑽♦')
('A♣', '⑽♣')
('K♠', 'K♥')
('K♠', 'K♦')
('K♠', 'K♣')
('K♠', 'Q♠')
('K♠', 'Q♥')
('K♠', 'Q♦')
('K♠', 'Q♣')

**Exercise:** Write a generator function called `combinations` that takes an iterator and and integer, `r`, as arguments. It should generate tuples that represent all *sorted* subsets of the elements in the iterator with size `r` and no duplicates.

Test your function by generating and printing all hands with two distinct cards.
Then use `ilen` to count how many there are, and confirm that it's `52 * 51 / 2`.

In [22]:
def combinations(it, r):
  combinations = []
  for hand in permutations(it, r):
    if sorted(hand) not in combinations:
      combinations.append(sorted(hand))
      yield hand

The `itertools` library provides a function called `combinations` that does the same thing.

In [17]:
import itertools

it = card_generator(ranks, suits)
ilen(itertools.combinations(it, 2))

1326

## Generating hands

We can use `combinations ` to write a generator that yields all valid hands with `n` playing cards, where "valid" means that the cards are in sorted order with no duplicates.

In [18]:
def hand_generator(n=2):
    it = card_generator(ranks, suits)
    for hand in combinations(it, n):
        yield hand

In [24]:
ilen(hand_generator(2))

1326

If you ever find yourself looping through an iterator and yielding all of the elements, you can simplify the code using `yield from`.

In [23]:
def hand_generator(n=2):
    it = card_generator(ranks, suits)
    yield from combinations(it, n)

In [25]:
ilen(hand_generator(2))

1326

Now let's see how many hands there are with 3, 4, and (maybe) 5 cards.

In [ ]:
ilen(hand_generator(3))

In [ ]:
ilen(hand_generator(4))

I'm not patient enough to let this one finish.

In [ ]:
# ilen(hand_generator(5))

But if we only care about the number of combinations, we can use [`math.comb`](https://docs.python.org/3/library/math.html).

In [ ]:
from math import comb

comb(52, 5)

## How many flushes?

In poker, a "flush" is a hand where all cards have the same suit.
To check whether a hand is a flush, it is convenient to extract the suit part of the cards and make a set.

In [26]:
it = hand_generator(4)
hand = next(it)
hand

('A♠', 'A♥', 'A♦', 'A♣')

In [ ]:
set(card[1] for card in hand)

**Exercise:** Write a function called `is_flush` that takes a hand as an argument and returns `True` if all cards are the same suit.

Then write a generator function called `flush_generator` that takes an integer `n` and return all hands with `n` cards that are flushes.

What fraction of hands with 3, 4, and 5 cards are flushes?

In [29]:
def is_flush (hand):
  return len(set(card[1] for card in hand)) == 1

In [31]:
def flush_generator (n):
  it = hand_generator(n)
  for hand in it:
    if is_flush(hand):
      yield hand

In [32]:
ilen(flush_generator(3))/ilen(hand_generator(3))

0.05176470588235294

## Write your own product

So far we've been using `itertools.product`, but in the same way we wrote `permutations` and `combinations`, we can write our own `product`.

If there are only two iterators, we can do it with nested `for` loops.

In [35]:
def product2(it1, it2):
    for x in it1:
        for y in it2:
            yield x, y

So we can generate the cards like this.

In [36]:
for t in product2(ranks, suits):
    card = ''.join(t)
    print(card, end=' ')

A♠ A♥ A♦ A♣ K♠ K♥ K♦ K♣ Q♠ Q♥ Q♦ Q♣ J♠ J♥ J♦ J♣ ⑽♠ ⑽♥ ⑽♦ ⑽♣ 9♠ 9♥ 9♦ 9♣ 8♠ 8♥ 8♦ 8♣ 7♠ 7♥ 7♦ 7♣ 6♠ 6♥ 6♦ 6♣ 5♠ 5♥ 5♦ 5♣ 4♠ 4♥ 4♦ 4♣ 3♠ 3♥ 3♦ 3♣ 2♠ 2♥ 2♦ 2♣ 

Now, we might be tempted to write two-card hands like this.

In [39]:
it1 = card_generator(ranks, suits)
it2 = card_generator(ranks, suits)

for hand in product2(it1, it2):
    print(hand, end=' ')

('A♠', 'A♠') ('A♠', 'A♥') ('A♠', 'A♦') ('A♠', 'A♣') ('A♠', 'K♠') ('A♠', 'K♥') ('A♠', 'K♦') ('A♠', 'K♣') ('A♠', 'Q♠') ('A♠', 'Q♥') ('A♠', 'Q♦') ('A♠', 'Q♣') ('A♠', 'J♠') ('A♠', 'J♥') ('A♠', 'J♦') ('A♠', 'J♣') ('A♠', '⑽♠') ('A♠', '⑽♥') ('A♠', '⑽♦') ('A♠', '⑽♣') ('A♠', '9♠') ('A♠', '9♥') ('A♠', '9♦') ('A♠', '9♣') ('A♠', '8♠') ('A♠', '8♥') ('A♠', '8♦') ('A♠', '8♣') ('A♠', '7♠') ('A♠', '7♥') ('A♠', '7♦') ('A♠', '7♣') ('A♠', '6♠') ('A♠', '6♥') ('A♠', '6♦') ('A♠', '6♣') ('A♠', '5♠') ('A♠', '5♥') ('A♠', '5♦') ('A♠', '5♣') ('A♠', '4♠') ('A♠', '4♥') ('A♠', '4♦') ('A♠', '4♣') ('A♠', '3♠') ('A♠', '3♥') ('A♠', '3♦') ('A♠', '3♣') ('A♠', '2♠') ('A♠', '2♥') ('A♠', '2♦') ('A♠', '2♣') 

But that doesn't work; it only generates the first 52 pairs.
Before you go on, see if you can figure out why.

We can solve this problem by making each iterator into a tuple; then we can loop through them more than once.
The price we pay is that we have to store all of the elements of the iterators.

In [ ]:
def product2(it1, it2):
    t1 = tuple(it1)
    t2 = tuple(it2)
    for x in t1:
        for y in t2:
            yield x, y

This version of `product2` works if the arguments are iterators.

In [ ]:
it1 = card_generator(ranks, suits)
it2 = card_generator(ranks, suits)

for hand in product2(it1, it2):
    print(hand)

In [ ]:
it1 = card_generator(ranks, suits)
it2 = card_generator(ranks, suits)

ilen(product2(it1, it2))

Now let's take it up a notch. What if you want the product of more than two iterators.
The version of `product` we got from `itertools` can handle this case.

In [41]:
import itertools

for pair in itertools.product(range(2), range(3), range(4)):
    print(pair, end=' ')

(0, 0, 0) (0, 0, 1) (0, 0, 2) (0, 0, 3) (0, 1, 0) (0, 1, 1) (0, 1, 2) (0, 1, 3) (0, 2, 0) (0, 2, 1) (0, 2, 2) (0, 2, 3) (1, 0, 0) (1, 0, 1) (1, 0, 2) (1, 0, 3) (1, 1, 0) (1, 1, 1) (1, 1, 2) (1, 1, 3) (1, 2, 0) (1, 2, 1) (1, 2, 2) (1, 2, 3) 

**Exercise:** Write a generator function that takes an arbitrary number of iterables and yields their Cartesian product. Compare the results to `itertools.product`.

Hint: I found it easiest to write this recursively.

In [42]:
# prompt: Write a generator function that takes an arbitrary number of iterables and yields their Cartesian product

def my_product(*iterables):
    if not iterables:
        yield ()
        return

    first, rest = iterables[0], iterables[1:]

    for item in first:
        for prod_of_rest in my_product(*rest):
            yield (item,) + prod_of_rest

# Test cases
for pair in my_product(range(2), range(3), range(4)):
    print(pair, end=' ')

(0, 0, 0) (0, 0, 1) (0, 0, 2) (0, 0, 3) (0, 1, 0) (0, 1, 1) (0, 1, 2) (0, 1, 3) (0, 2, 0) (0, 2, 1) (0, 2, 2) (0, 2, 3) (1, 0, 0) (1, 0, 1) (1, 0, 2) (1, 0, 3) (1, 1, 0) (1, 1, 1) (1, 1, 2) (1, 1, 3) (1, 2, 0) (1, 2, 1) (1, 2, 2) (1, 2, 3) 

In [46]:
def product_fn(*iterables):
  if not iterables:
    yield()
    return
  for item in iterables[0]:
    for it in product_fn(*iterables[1:]):
      yield (item,) + it
for pair in product_fn(range(2), range(3), range(4)):
    print(pair, end=' ')

(0, 0, 0) (0, 0, 1) (0, 0, 2) (0, 0, 3) (0, 1, 0) (0, 1, 1) (0, 1, 2) (0, 1, 3) (0, 2, 0) (0, 2, 1) (0, 2, 2) (0, 2, 3) (1, 0, 0) (1, 0, 1) (1, 0, 2) (1, 0, 3) (1, 1, 0) (1, 1, 1) (1, 1, 2) (1, 1, 3) (1, 2, 0) (1, 2, 1) (1, 2, 2) (1, 2, 3) 

In [47]:
%timeit product_fn(range(2), range(3), range(4))

599 ns ± 19 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [48]:
%timeit my_product(range(2), range(3), range(4))

746 ns ± 199 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [49]:
%timeit itertools.product(range(2), range(3), range(4))

676 ns ± 58.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


*Data Structures and Information Retrieval in Python*

Copyright 2021 Allen Downey

License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)